In [566]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re # regex
from urllib.parse import urlparse, parse_qs # urlparse & parse_qs
from scipy.stats import entropy # Cho url_entropy ở

In [567]:
path = '/kaggle/input/labeldata/normal_labeled.log'

In [568]:
import re
from typing import Union, Iterable, List, Dict, Optional
import pandas as pd

# ──────────────────────────────────────────────────────────────
# 1) Pre-compile regex (Nginx / Apache “combined” format)
#    - STRICT   : có dấu " quanh request + referrer + user-agent
#    - FALLBACK : thiếu hoặc hỏng dấu " (bắt thoáng hơn)
# ──────────────────────────────────────────────────────────────
NGINX_REGEX_STRICT = re.compile(
    r'(?P<ip>\S+)\s+-\s+-\s+'                              # IP - -
    r'\[(?P<timestamp>[^\]]+)]\s+'                         # [timestamp]
    r'"(?P<method>[A-Z]+)\s+'                              # "METHOD␣
    r'(?P<url>.+?)\s+'                                     # URL (non-greedy)
    r'(?P<protocol>[A-Z]+/\d(?:\.\d)?)"\s+'                # PROTOCOL"
    r'(?P<status>\d{3}|-)\s+'                              # status
    r'(?P<size>\d+|-)\s+'                                  # size
    r'"(?P<referrer>[^"]*)"\s+'                            # "referrer"
    r'"(?P<user_agent>[^"]*)"'                            # "user-agent"
    r'(?:[ \t]+(?P<label>[01]))?$',        #  ← thêm nhóm label tuỳ chọn
    flags=re.IGNORECASE,
)

NGINX_REGEX_FALLBACK = re.compile(
    r'(?P<ip>\S+)\s+-\s+-\s+'                              # IP - -
    r'\[(?P<timestamp>[^\]]+)]\s+'                         # [timestamp]
    r'(?P<method>[A-Z]+)\s+'                               # METHOD
    r'(?P<url>.+?)\s+'                                     # URL
    r'(?P<protocol>[A-Z]+/\d(?:\.\d)?)\s+'                 # PROTOCOL
    r'(?P<status>\d{3}|-)\s+'                              # status
    r'(?P<size>\d+|-)\s+'                                  # size
    r'(?P<referrer>\S+|-)\s+'                              # referrer (không quotes)
    r'(?P<user_agent>.+)'                                 # user-agent (còn lại)
    r'(?:[ \t]+(?P<label>[01]))?$',        #  ← thêm nhóm label tuỳ chọn

    flags=re.IGNORECASE,
)

# Gộp thành tuple để lần lượt thử
NGINX_COMBINED_PATTERNS = (NGINX_REGEX_STRICT, NGINX_REGEX_FALLBACK)

# ──────────────────────────────────────────────────────────────
# 2) Tiện ích: loại bỏ ký tự control (nếu log bị lẫn \x00 …)
# ──────────────────────────────────────────────────────────────
def strip_control(s: str) -> str:
    """Remove leading control chars (0x00–0x1F) ở đầu dòng."""
    return re.sub(r'^[\x00-\x1F]+', "", s)

# ──────────────────────────────────────────────────────────────
# 3) Hàm wrapper parse_nginx_log
# ──────────────────────────────────────────────────────────────
def parse_nginx_log(
    source: Union[str, Iterable[str]],
    patterns: Iterable[re.Pattern] = NGINX_COMBINED_PATTERNS,
    as_dataframe: bool = True,
    encoding: Optional[str] = "utf-8",
) -> Union[pd.DataFrame, List[Dict[str, str]]]:
    """
    Parse log Nginx / Apache (combined) thành list[dict] hoặc pandas.DataFrame.

    Args:
        source (str | Iterable[str]):
            • Chuỗi đường dẫn file, hoặc
            • Iterable (list, generator, ...) các dòng log.
        patterns (Iterable[re.Pattern]): Danh sách regex sẽ thử lần lượt.
        as_dataframe (bool): True -> trả về DataFrame, False -> list[dict].
        encoding (str | None): Encoding khi mở file (nếu source là path).

    Returns:
        pandas.DataFrame | list[dict]
    """
    # 1) Lấy iterator dòng log
    if isinstance(source, str):                # truyền path
        fh = open(source, "r", encoding=encoding, errors="replace")
        lines = fh
        close_file = True
    else:                                      # iterable dòng
        lines = source
        close_file = False

    # 2) Parse
    parsed: List[Dict[str, str]] = []
    for raw_line in lines:
        line = strip_control(raw_line.rstrip("\n"))
        for pat in patterns:
            m = pat.match(line)
            if m:
                parsed.append(m.groupdict())
                break                          # matched → sang dòng kế
        # nếu muốn ghi lại MISS, thêm else: missed.append(line)

    # 3) Đóng file nếu cần
    if close_file:
        fh.close()

    # 4) Trả kết quả
    return pd.DataFrame(parsed) if as_dataframe else parsed

In [569]:
# stores output in parsed_log.csv
import pandas as pd
df = parse_nginx_log(path)

In [570]:
df.isna().sum()/len(df)

ip            0.00000
timestamp     0.00000
method        0.00000
url           0.00000
protocol      0.00000
status        0.00000
size          0.00000
referrer      0.00000
user_agent    0.00000
label         0.00061
dtype: float64

In [571]:
mask = df['label'].isna() | (df['label'].astype(str).str.strip() == '')
rows_without_label = df[mask]

# Xem nhanh vài dòng lỗi
print(rows_without_label.head())

# Hoặc ghi ra file để soi kỹ
rows_without_label.to_csv('no_label_rows.csv', index=False)
print(f"Số dòng thiếu label: {len(rows_without_label)}")

                   ip                   timestamp method  \
100159   62.82.191.64  04/Oct/2024:07:39:55 +0000   POST   
100196  106.62.177.91  25/Dec/2024:15:45:47 +0000  PATCH   
100241  106.62.177.91  25/Dec/2024:15:46:43 +0000   POST   
100439    50.79.137.2  05/Mar/2025:11:01:09 +0000  PATCH   
100451    50.79.137.2  05/Mar/2025:11:00:41 +0000   POST   

                                                                 url  \
100159  /search?q=3c7363726970743e616c6572742831293c2f7363726970743e   
100196              /login?username=admin&password=1' OR sleep(5)--+   
100241                                              /run?cmd=;ls -la   
100439                                                        /admin   
100451                   /run?cmd=7c20636174202f6574632f706173737764   

        protocol status  size         referrer              user_agent label  
100159  HTTP/1.0    403  2947  http://evil.com  sqlmap/1.4.12.1#dev\t1  None  
100196  HTTP/1.0    200  3436  http://evil.com  

In [572]:
df.sample(5)

,ip,timestamp,method,url,protocol,status,size,referrer,user_agent,label
89025,203.0.113.244,10/Jun/2025:01:36:57 +0000,GET,/home,HTTP/1.1,200,1887,https://example.com/blog/blog,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:{0}.0) Gecko/20100101 Firefox/{0}.0,0
93978,10.0.0.13,08/Jun/2025:23:34:55 +0000,POST,/search?q=camera,HTTP/1.1,200,2347,https://www.google.com/,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),0
26100,203.0.113.133,09/Jun/2025:07:08:16 +0000,POST,/about,HTTP/1.1,500,1877,https://example.com/,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),0
12161,172.16.0.43,08/Jun/2025:02:20:32 +0000,GET,/images/logo.png,HTTP/1.1,200,3252,https://www.google.com/search?q=shop,Twitterbot/1.0,0
40002,172.16.0.116,10/Jun/2025:15:46:12 +0000,GET,/search?q=camera,HTTP/1.1,200,4307,https://www.bing.com/,"Mozilla/5.0 (iPhone; CPU iPhone OS {0}_{1} like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/{2}.0 Mobile/15E148 Safari/604.1",0


## Time-based feature handling

In [573]:
def timestamp_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Nhận vào một DataFrame chứa cột 'timestamp' và trả về DataFrame
    đã được bổ sung đầy đủ các feature về thời gian.
    """
    df['timestamp_dt'] = pd.to_datetime(df['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')
    
    df = df.sort_values('timestamp_dt').reset_index(drop=True)

    df['hour_of_day'] = df['timestamp_dt'].dt.hour
    df['day_of_week'] = df['timestamp_dt'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

    def get_part_of_day(hour):
        if 5 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 17:
            return 'afternoon'
        elif 17 <= hour < 21:
            return 'evening'
        else:
            return 'night'
    df['part_of_day'] = df['hour_of_day'].apply(get_part_of_day)

    df['hour_sin'] = np.sin(2 * np.pi * df['hour_of_day'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour_of_day'] / 24)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    df['time_since_last_event'] = df['timestamp_dt'].diff().dt.total_seconds().fillna(0)
    df.drop(columns=["timestamp_dt"], inplace=True)
    df.drop(columns=["timestamp"], inplace=True)

    return df

In [574]:
df = timestamp_features(df)
df.sample(5)

,ip,method,url,protocol,status,size,referrer,user_agent,label,hour_of_day,day_of_week,is_weekend,part_of_day,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,time_since_last_event
84686,10.0.0.248,GET,/search?q=camera,HTTP/1.1,200,2227,https://example.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:{0}.0) Gecko/20100101 Firefox/{0}.0,0,6,1,0,morning,1.000000,6.123234e-17,0.781831,0.623490,5.0
49601,101.102.99.157,POST,/wp-login.php,HTTP/1.1,200,1818,https://www.google.com/search?q=news,Twitterbot/1.0,0,11,5,1,morning,0.258819,-9.659258e-01,-0.974928,-0.222521,3.0
70231,101.102.220.102,POST,/wp-login.php,HTTP/1.1,404,4561,https://twitter.com/,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_{0}_{1}) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/{2}.0 Safari/605.1.15",0,2,0,0,night,0.500000,8.660254e-01,0.000000,1.000000,3.0
81444,45.67.63.18,GET,/product/715/detail,HTTP/1.1,500,2061,https://www.facebook.com/,Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm),0,0,1,0,night,0.000000,1.000000e+00,0.781831,0.623490,3.0
104157,203.0.113.100,GET,/user/680,HTTP/1.1,404,4925,https://www.google.com/search?q=news,Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm),0,19,2,0,evening,-0.965926,2.588190e-01,0.974928,-0.222521,18.0


# Feature extraction process

## URL Feature handling

## URL Suspicious Patterns

In [575]:
import pandas as pd
import re
from urllib.parse import unquote_plus
import base64

# ==============================================================================
# HÀM PHÂN TÍCH CUỐI CÙNG (DANH SÁCH PATTERN ĐÃ ĐƯỢC DỌN DẸP)
# ==============================================================================
def final_url_analyzer(url_string: str) -> tuple:
    """
    Hàm cuối cùng, với danh sách pattern đã được dọn dẹp và sửa lỗi.
    """
    # --- BỘ QUY TẮC ĐÃ ĐƯỢC TINH GỌN VÀ SỬA LỖI ---
    patterns = [
        # --- SQL Injection ---
        r'(union\s+select)',
        r'(select\s+.*\s+from)',
        r'(insert\s+into)',
        r'(delete\s+from)',
        r'(drop\s+table)',
        r'(--|#|\/\*|;\s*--)',
        r'(load_file\s*\()',
        r'(information_schema\.)',
        r'(pg_sleep|waitfor\s+delay|sleep|benchmark)\s*\(',
        r'(xp_cmdshell)',
        # <<< Sửa lỗi cho log #7: bắt OR '1'='1 và các dạng tương tự >>>
        r"(?:'|\")\s*or\s+(?:'|\")?.+?(?:'|\")?\s*=\s*(?:'|\")?.+?(?:'|\")?",

        # --- XSS ---
        r'(<script)',
        r'(<iframe)',
        r'(<svg)',
        r'(<img\s+[^>]*src\s*=\s*[\'"]?javascript:)',
        r'(on(error|load|mouseover)\s*=)',
        r'(eval\s*\()',
        r'(document\.cookie)',
        r'(alert\s*\()',

        # --- Path Traversal & File Inclusion ---
        r'(\.\.\/|\.\.\\|%2e%2e|%c0%ae)',
        r'(etc\/(passwd|shadow))',
        r'(proc\/(self|environ))',
        r'(boot\.ini|win\.ini)',
        r'(php|file|data):\/\/',

        # --- Command Injection ---
        r'(&&|;|\||`|\$\()',
        r'(\b(cat|ls|whoami|id|wget|curl|bash|sh|cmd)\s+)',
        r'(\/bin\/(ba)?sh)',
    ]
    combined_pattern = re.compile('|'.join(patterns), re.IGNORECASE)

    strings_to_check = set()
    try:
        strings_to_check.add(url_string)
        decoded_url = unquote_plus(unquote_plus(url_string))
        strings_to_check.add(decoded_url)
    except:
        decoded_url = url_string

    # Tách các phần và thử decode hex/base64
    parts = re.split(r'[=,&;/?]', decoded_url)
    for part in parts:
        part = part.strip()
        if len(part) < 4:
            continue
        # hex
        try:
            if all(c in '0123456789abcdefABCDEF' for c in part) and len(part) % 2 == 0:
                strings_to_check.add(bytes.fromhex(part).decode('utf-8', 'ignore'))
        except:
            pass
        # base64
        try:
            missing_padding = len(part) % 4
            if missing_padding:
                part += '=' * (4 - missing_padding)
            strings_to_check.add(base64.b64decode(part).decode('utf-8', 'ignore'))
        except:
            pass

    # Quét từng chuỗi
    for text in strings_to_check:
        match = combined_pattern.search(text)
        if match:
            return 1

    return 0



In [576]:
df[df[['url', 'method', 'protocol']].isnull().any(axis=1)]['url']


Series([], Name: url, dtype: object)

In [577]:
df[['is_suspicious']] = df['url'].apply(lambda x: pd.Series(final_url_analyzer(x)))
df

,ip,method,url,protocol,status,size,referrer,user_agent,label,hour_of_day,day_of_week,is_weekend,part_of_day,hour_sin,hour_cos,day_of_week_sin,day_of_week_cos,time_since_last_event,is_suspicious
0,55.11.206.13,GET,/search?q=PGJvZHkgb25sb2FkPWFsZXJ0KCdYU1MnKT4=,HTTP/1.0,500,3780,http://evil.com,sqlmap/1.4.12.1#dev,1,6,6,1,morning,1.000000,6.123234e-17,-0.781831,0.623490,0.0,1
1,55.11.206.13,GET,/search?q=PGJvZHkgb25sb2FkPWFsZXJ0KCdYU1MnKT4=,HTTP/2.0,499,604,-,sqlmap/1.4.12.1#dev,1,6,6,1,morning,1.000000,6.123234e-17,-0.781831,0.623490,6.0,1
2,55.11.206.13,DELETE,/search?q=<a href='javascript:alert(1)'>click</a>,HTTP/1.0,404,1730,-,sqlmap/1.4.12.1#dev,1,6,6,1,morning,1.000000,6.123234e-17,-0.781831,0.623490,1.0,1
3,55.11.206.13,DELETE,/run?cmd=%2526%2526%2520whoami,HTTP/1.0,403,211,-,sqlmap/1.4.12.1#dev,1,6,6,1,morning,1.000000,6.123234e-17,-0.781831,0.623490,0.0,1
4,55.11.206.13,GET,/search?q=PGJvZHkgb25sb2FkPWFsZXJ0KCdYU1MnKT4=,HTTP/2.0,405,451,http://evil.com,sqlmap/1.4.12.1#dev,1,6,6,1,morning,1.000000,6.123234e-17,-0.781831,0.623490,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104992,172.16.0.31,GET,/about,HTTP/1.1,200,3880,https://www.linkedin.com/,"Mozilla/5.0 (iPhone; CPU iPhone OS {0}_{1} like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/{2}.0 Mobile/15E148 Safari/604.1",0,21,2,0,night,-0.707107,7.071068e-01,0.974928,-0.222521,1.0,0
104993,198.51.100.178,GET,/category/655/items,HTTP/1.1,200,1363,https://www.bing.com/,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),0,21,2,0,night,-0.707107,7.071068e-01,0.974928,-0.222521,6.0,0
104994,203.0.113.250,POST,/product/720/detail,HTTP/1.1,200,465,https://example.com/,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{0}.0.{1}.{2} Safari/537.36",0,21,2,0,night,-0.707107,7.071068e-01,0.974928,-0.222521,4.0,0
104995,198.51.100.133,POST,/sitemap.xml,HTTP/1.1,404,2857,https://twitter.com/,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_{0}_{1}) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/{2}.0 Safari/605.1.15",0,21,2,0,night,-0.707107,7.071068e-01,0.974928,-0.222521,12.0,0


## User Agent Features

In [578]:
from user_agents import parse

def calculate_entropy(text_string: str) -> float:
    """
    Tính entropy của chuỗi ký tự (dựa trên xác suất xuất hiện ký tự).
    """
    import math
    from collections import Counter

    if not text_string:
        return 0.0

    counts = Counter(text_string)
    total = len(text_string)
    entropy = -sum((count / total) * math.log2(count / total) for count in counts.values())
    return entropy


def user_agent_features(df):
    """
    Thêm các đặc trưng liên quan đến User-Agent vào DataFrame đầu vào.
    """
    df = df.copy()  # tránh tác động trực tiếp

    df['ua_parsed'] = df['user_agent'].astype(str).apply(parse)

    # 1. Trình duyệt (Browser Family)
    df['ua_browser_family'] = df['ua_parsed'].apply(lambda ua: ua.browser.family)

    # 2. Phiên bản trình duyệt (Major Version)
    df['ua_browser_version_major'] = df['ua_parsed'].apply(lambda ua: ua.browser.version[0] if ua.browser.version else None)

    # 3. Hệ điều hành (OS Family)
    df['ua_os_family'] = df['ua_parsed'].apply(lambda ua: ua.os.family)

    # 4. Phiên bản hệ điều hành (Major Version)
    df['ua_os_version_major'] = df['ua_parsed'].apply(lambda ua: ua.os.version[0] if ua.os.version else None)

    # 5. Thiết bị (Device Family/Brand)
    df['ua_device_family'] = df['ua_parsed'].apply(lambda ua: ua.device.family)
    df['ua_device_brand'] = df['ua_parsed'].apply(lambda ua: ua.device.brand)

    # 6-10. Các flag nhận diện thiết bị
    df['ua_is_bot'] = df['ua_parsed'].apply(lambda ua: int(ua.is_bot))
    df['ua_is_mobile'] = df['ua_parsed'].apply(lambda ua: int(ua.is_mobile))
    df['ua_is_tablet'] = df['ua_parsed'].apply(lambda ua: int(ua.is_tablet))
    df['ua_is_pc'] = df['ua_parsed'].apply(lambda ua: int(ua.is_pc))
    df['ua_is_touch_capable'] = df['ua_parsed'].apply(lambda ua: int(ua.is_touch_capable))

    # 11. Độ dài chuỗi User-Agent
    df['ua_length'] = df['user_agent'].astype(str).apply(len)

    # 12. Entropy của User-Agent
    df['ua_entropy'] = df['user_agent'].astype(str).apply(calculate_entropy)

    tools = ['sqlmap', 'curl', 'wget', 'nmap', 'nikto', 'fuzz', 'hydra']
    df['ua_is_tool'] = df['user_agent'].str.lower().apply(lambda ua: any(tool in ua for tool in tools)).astype(int)

    # Dọn bộ nhớ
    df.drop('ua_parsed', axis=1, inplace=True)

    return df

In [579]:
df = user_agent_features(df)
df.sample(5)

,ip,method,url,protocol,status,size,referrer,user_agent,label,hour_of_day,...,ua_device_family,ua_device_brand,ua_is_bot,ua_is_mobile,ua_is_tablet,ua_is_pc,ua_is_touch_capable,ua_length,ua_entropy,ua_is_tool
94630,101.102.92.199,POST,/category/36/items,HTTP/1.1,200,126,https://www.linkedin.com/,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{0}.0.{1}.{2} Safari/537.36",0,1,...,Other,None,0,0,0,1,0,115,5.226284,0
70309,172.16.0.251,POST,/admin/login,HTTP/1.1,200,3462,https://www.bing.com/,Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm),0,2,...,Spider,Spider,1,0,0,0,0,71,4.542884,0
100646,192.168.1.158,POST,/dashboard,HTTP/1.1,200,4420,https://www.google.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:{0}.0) Gecko/20100101 Firefox/{0}.0,0,13,...,Other,None,0,0,0,1,0,80,4.777384,0
38474,10.0.0.148,GET,/sitemap.xml,HTTP/1.1,200,363,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:{0}.0) Gecko/20100101 Firefox/{0}.0,0,13,...,Other,None,0,0,0,1,0,80,4.777384,0
47790,101.102.127.46,POST,/sitemap.xml,HTTP/1.1,200,3880,https://www.google.com/search?q=shop,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/{0}.0.{1}.{2} Safari/537.36",0,7,...,Other,None,0,0,0,1,0,115,5.226284,0


In [580]:
cols_to_show_ua = [
    'user_agent', 'ua_browser_family', 'ua_os_family', 'ua_device_brand',
    'ua_is_bot', 'ua_is_mobile', 'ua_is_pc', 'ua_length', 'ua_entropy', 'ua_'
]

print("\n--- Features từ User-Agent ---")
print(df[[col for col in cols_to_show_ua if col in df.columns]].head())


--- Features từ User-Agent ---
            user_agent ua_browser_family ua_os_family ua_device_brand  \
0  sqlmap/1.4.12.1#dev             Other        Other            None   
1  sqlmap/1.4.12.1#dev             Other        Other            None   
2  sqlmap/1.4.12.1#dev             Other        Other            None   
3  sqlmap/1.4.12.1#dev             Other        Other            None   
4  sqlmap/1.4.12.1#dev             Other        Other            None   

   ua_is_bot  ua_is_mobile  ua_is_pc  ua_length  ua_entropy  
0          0             0         0         19    3.747413  
1          0             0         0         19    3.747413  
2          0             0         0         19    3.747413  
3          0             0         0         19    3.747413  
4          0             0         0         19    3.747413  


In [583]:
def status_features(df):
    """
    Trích xuất các feature từ cột 'status' và 'size' trong log web.

    Args:
        df (pd.DataFrame): DataFrame chứa ít nhất 2 cột: 'status' (int), 'size' (int)

    Returns:
        pd.DataFrame: DataFrame gốc kèm thêm các cột đặc trưng mới.
    """
    # Kiểm tra cột trước
    if 'status' not in df.columns or 'size' not in df.columns:
        raise ValueError("DataFrame cần có cột 'status' và 'size'.")

    df = df.copy()
    df['status'] = pd.to_numeric(df['status'], errors='coerce').fillna(0).astype(int)
    df['size'] = pd.to_numeric(df['size'], errors='coerce').fillna(0).astype(int)
    # 1. 4xx - lỗi phía client
    df['status_is_client_error'] = df['status'].apply(lambda x: 1 if 400 <= x < 500 else 0)

    # 2. 5xx - lỗi phía server
    df['status_is_server_error'] = df['status'].apply(lambda x: 1 if 500 <= x < 600 else 0)

    # 3. Lỗi nói chung
    df['status_is_error'] = ((df['status_is_client_error'] == 1) | (df['status_is_server_error'] == 1)).astype(int)

    # 4. Thành công (2xx)
    df['status_is_success'] = df['status'].apply(lambda x: 1 if 200 <= x < 300 else 0)

    # 5. Redirect (3xx)
    df['status_is_redirect'] = df['status'].apply(lambda x: 1 if 300 <= x < 400 else 0)

    # 6. Response size bằng 0
    df['size_is_zero'] = df['size'].apply(lambda x: 1 if x == 0 else 0)

    return df


In [584]:
df = status_features(df)
df.sample(5)

,ip,method,url,protocol,status,size,referrer,user_agent,label,hour_of_day,...,ua_is_touch_capable,ua_length,ua_entropy,ua_is_tool,status_is_client_error,status_is_server_error,status_is_error,status_is_success,status_is_redirect,size_is_zero
56945,203.0.113.143,GET,/,HTTP/1.1,200,1606,https://www.facebook.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:{0}.0) Gecko/20100101 Firefox/{0}.0,0,1,...,0,80,4.777384,0,0,0,0,1,0,0
63045,192.168.1.108,POST,/category/344/items,HTTP/1.1,200,1753,https://www.google.com/search?q=tech,Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html),0,12,...,0,72,4.457460,0,0,0,0,1,0,0
1010,79.189.139.73,PATCH,/login?username=admin&password=1%2527%2520OR%2520sleep%25285%2529--%252B,HTTP/2.0,200,1915,-,<script>alert('UA')</script>,1,6,...,0,28,3.967720,0,0,0,0,1,0,0
34740,198.51.100.152,GET,/dashboard,HTTP/1.1,200,781,https://www.google.com/search?q=blog,"Mozilla/5.0 (iPhone; CPU iPhone OS {0}_{1} like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/{2}.0 Mobile/15E148 Safari/604.1",0,6,...,1,139,5.126522,0,0,0,0,1,0,0
95485,10.0.0.178,POST,/index.html,HTTP/1.1,200,4142,https://example.com/,"Mozilla/5.0 (iPhone; CPU iPhone OS {0}_{1} like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/{2}.0 Mobile/15E148 Safari/604.1",0,3,...,1,139,5.126522,0,0,0,0,1,0,0


In [585]:
from urllib.parse import urlparse

def referrer_features(df):
    df = df.copy()
    
    df['referrer_len'] = df['referrer'].astype(str).apply(len)
    df['referrer_entropy'] = df['referrer'].astype(str).apply(calculate_entropy)
    df['referrer_is_empty'] = df['referrer'].apply(lambda x: 1 if x == '-' or pd.isna(x) or str(x).strip() == '' else 0)

    def get_referrer_domain(ref):
        ref_str = str(ref)
        if pd.isna(ref) or ref_str == '-' or not ref_str.startswith('http'):
            return 'none'
        try:
            parsed = urlparse(ref_str)
            return parsed.netloc.lower() if parsed.netloc else 'unknown_format_but_not_empty'
        except:
            return 'parse_error'

    df['referrer_domain'] = df['referrer'].apply(get_referrer_domain)
    df['referrer_is_external_or_valid'] = df['referrer_domain'].apply(
        lambda x: 0 if x in ['none', 'parse_error', 'unknown_format_but_not_empty'] else 1
    )

    return df


In [586]:
df = referrer_features(df)
df.sample(3)

,ip,method,url,protocol,status,size,referrer,user_agent,label,hour_of_day,...,status_is_server_error,status_is_error,status_is_success,status_is_redirect,size_is_zero,referrer_len,referrer_entropy,referrer_is_empty,referrer_domain,referrer_is_external_or_valid
2641,143.222.96.239,POST,/debug,HTTP/1.1,200,2622,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64),1,8,...,0,0,1,0,0,1,-0.000000,1,none,0
23935,10.0.0.213,GET,/images/logo.png,HTTP/1.1,200,2644,https://example.com/blog/blog,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_{0}_{1}) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/{2}.0 Safari/605.1.15",0,9,...,0,0,1,0,0,29,3.840403,0,example.com,1
63143,192.168.1.43,POST,/admin,HTTP/1.1,200,3788,https://example.com/blog/news,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_{0}_{1}) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/{2}.0 Safari/605.1.15",0,13,...,0,0,1,0,0,29,4.004364,0,example.com,1


In [ ]:
# def behavior_features(df):
#     df = df.copy()

#     df['ip_request_count_total'] = df.groupby('ip')['ip'].transform('count')
#     df['ip_error_rate'] = df.groupby('ip')['status_is_error'].transform('mean')
#     df['ip_avg_query_count'] = df.groupby('ip')['url_query_count'].transform('mean')

#     return df


In [ ]:
# df = behavior_features(df)
# df.sample(3)

In [ ]:
df.info()

## Data preprocessing 2

In [ ]:

cat_col = df.select_dtypes(include=["object", "category"]).columns.tolist()

num_col = df.select_dtypes(include=["number"]).columns.tolist()


In [ ]:
len(num_col)

In [ ]:
len(cat_col)

In [ ]:
cat_col

In [ ]:
# Đã trích xuất nên drop
#df.drop(columns=["ip", "url", "referrer"], inplace=True)
df.drop(columns=['user_agent'], inplace=True)

In [ ]:
df.drop(columns=["ip", "url", "referrer"], inplace=True)


In [588]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104997 entries, 0 to 104996
Data columns (total 44 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   ip                             104997 non-null  object 
 1   method                         104997 non-null  object 
 2   url                            104997 non-null  object 
 3   protocol                       104997 non-null  object 
 4   status                         104997 non-null  int64  
 5   size                           104997 non-null  int64  
 6   referrer                       104997 non-null  object 
 7   user_agent                     104997 non-null  object 
 8   label                          104933 non-null  object 
 9   hour_of_day                    104997 non-null  int32  
 10  day_of_week                    104997 non-null  int32  
 11  is_weekend                     104997 non-null  int64  
 12  part_of_day                   

In [589]:
df.isna().sum()/len(df)

ip                               0.000000
method                           0.000000
url                              0.000000
protocol                         0.000000
status                           0.000000
size                             0.000000
referrer                         0.000000
user_agent                       0.000000
label                            0.000610
hour_of_day                      0.000000
day_of_week                      0.000000
is_weekend                       0.000000
part_of_day                      0.000000
hour_sin                         0.000000
hour_cos                         0.000000
day_of_week_sin                  0.000000
day_of_week_cos                  0.000000
time_since_last_event            0.000000
is_suspicious                    0.000000
ua_browser_family                0.000000
ua_browser_version_major         0.761879
ua_os_family                     0.000000
ua_os_version_major              0.745602
ua_device_family                 0

In [591]:
df.drop(columns=['ua_device_brand', 'ua_browser_version_major', 'ua_os_version_major'], inplace=True)
df = df[~df['label'].isna()]

In [592]:
from catboost import CatBoostClassifier, Pool

# 1. Tách X, y
X = df.drop(columns=['label'])     # giả sử bạn đã có cột label
y = df['label']

# 2. Xác định các cột categorical (có kiểu object hoặc category)
cat_col = X.select_dtypes(include=['object', 'category']).columns.tolist()

# 3. Chia train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Tạo Pool để CatBoost hiểu categorical
train_pool = Pool(X_train, y_train, cat_features=cat_col)
test_pool = Pool(X_test, y_test, cat_features=cat_col)

# 5. Train
model = CatBoostClassifier(verbose=100, random_state=42)
model.fit(train_pool)

# 6. Predict
y_pred = model.predict(test_pool)

# 7. Đánh giá
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


Learning rate set to 0.06831
0:	learn: 0.4028940	total: 168ms	remaining: 2m 47s
100:	learn: 0.0000299	total: 10.1s	remaining: 1m 29s
200:	learn: 0.0000279	total: 19.9s	remaining: 1m 18s
300:	learn: 0.0000278	total: 29.9s	remaining: 1m 9s
400:	learn: 0.0000277	total: 38.2s	remaining: 57s
500:	learn: 0.0000276	total: 46.2s	remaining: 46s
600:	learn: 0.0000276	total: 54.8s	remaining: 36.4s
700:	learn: 0.0000275	total: 1m 2s	remaining: 26.7s
800:	learn: 0.0000275	total: 1m 10s	remaining: 17.5s
900:	learn: 0.0000274	total: 1m 17s	remaining: 8.57s
999:	learn: 0.0000274	total: 1m 26s	remaining: 0us
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     19987
           1       1.00      1.00      1.00      1000

    accuracy                           1.00     20987
   macro avg       1.00      1.00      1.00     20987
weighted avg       1.00      1.00      1.00     20987

